In [ ]:
from flask import Flask
from flask import request
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import GridSearchCV
app = Flask(__name__)

@app.route("/")
def micin():
    df = pd.read_csv('data.csv')
    df.head()
    df = df.drop(['ca','cholesterol','oldpeak','slope','restecg','jenis_sakit_dada'],axis = 1)
    dt = df.drop(['diagnosis'],axis = 1)
    df.shape,dt.shape

    xTrain, xTest, yTrain, yTest = train_test_split(dt, df['diagnosis'], test_size = 0.25, random_state = 0)

    xTrain.shape,yTrain.shape,xTest.shape,yTest.shape

    df.head()

    clf = svm.SVC(kernel='rbf', gamma= 0.001, C=10).fit(xTrain, yTrain)
    akurasiSvm = clf.score(xTrain, yTrain),clf.score(xTest, yTest)
# umur jenis_kelamin tekanan_darah gula_darah thalach exang thal
    coba= [[]]

    #coba = [[35,1,0,0,130,1,7],[38,0,105,0,166,0,0]]
    param = request.args.get('umur')
    coba[0].append(param)
    param = request.args.get('jk')
    coba[0].append(param)
    param = request.args.get('td')
    coba[0].append(param)
    param = request.args.get('gd')
    coba[0].append(param)
    param = request.args.get('thalac')
    coba[0].append(param)
    param = request.args.get('exang')
    coba[0].append(param)
    param = request.args.get('thl')
    coba[0].append(param)
    #prediksiSvm = clf.predict(coba)
    
    prediksiSvm = clf.predict(coba)

    def svc_param_selection(X, y, nfolds):
        Cs = [0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000]
        gammas = [0.0001,0.001, 0.01, 0.1, 1, 10, 100]
        param_grid = {'C': Cs, 'gamma' : gammas}
        grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
        grid_search.fit(X, y)
        return grid_search.best_params_,grid_search.score(X,y)

    svc_param_selection(dt,df['diagnosis'],10)

    from sklearn.tree import DecisionTreeClassifier
    tree = DecisionTreeClassifier(random_state = 0)
    tree.fit(xTrain,yTrain)

    akurasiTree = tree.score(xTrain,yTrain), tree.score(xTest,yTest)

    prediksiTree = tree.predict(coba)
    
    from sklearn.ensemble import GradientBoostingClassifier
    gb = GradientBoostingClassifier(n_estimators=20, learning_rate = 0.1, max_features=2, max_depth = 2, random_state = 0)
    gb.fit(xTest, yTest)
    akurasiGradient = gb.score(xTest,yTest)
    
    prediksiGradient = gb.predict(coba)
    return str(prediksiGradient)


if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
127.0.0.1 - - [15/Oct/2019 10:43:41] "GET /?umur=20&jk=1&td=90&gd=0&thalac=79&exang=0&thl=0 HTTP/1.1" 200 -
//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_

In [39]:
from werkzeug.wrappers import Request, Response
from flask import Flask
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import GridSearchCV

df = pd.read_csv('data.csv')
df.head()
df = df.drop(['ca','cholesterol','oldpeak','slope','restecg','jenis_sakit_dada'],axis = 1)
dt = df.drop(['diagnosis'],axis = 1)
df.shape,dt.shape

xTrain, xTest, yTrain, yTest = train_test_split(dt, df['diagnosis'], test_size = 0.25, random_state = 0)

xTrain.shape,yTrain.shape,xTest.shape,yTest.shape

df.head()

clf = svm.SVC(kernel='rbf', gamma= 0.001, C=10).fit(xTrain, yTrain)
akurasiSvm = clf.score(xTrain, yTrain),clf.score(xTest, yTest)

coba = [[35,1,0,0,130,1,7],[38,0,105,0,166,0,0]]
# umur jenis_kelamin tekanan_darah gula_darah thalach exang thal
prediksiSvm = clf.predict(coba)

def svc_param_selection(X, y, nfolds):
    Cs = [0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000]
    gammas = [0.0001,0.001, 0.01, 0.1, 1, 10, 100]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    return grid_search.best_params_,grid_search.score(X,y)

svc_param_selection(dt,df['diagnosis'],10)

from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state = 0)
tree.fit(xTrain,yTrain)

akurasiTree = tree.score(xTrain,yTrain), tree.score(xTest,yTest)

prediksiTree = tree.predict(coba)

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
clf.fit(dt, df['diagnosis'])  

akurasiRandom = clf.score(xTest,yTest);

from sklearn.ensemble import GradientBoostingClassifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=20, learning_rate = 0.1, max_features=2, max_depth = 2, random_state = 0)
    gb.fit(xTest, yTest)
    akurasiGradient = gb.score(xTest,yTest)
    print("Learning rate: ", learning_rate)
    print("Akurasi Gradient Boosting : ", akurasiGradient)

print('Akurasi SVM : ',akurasiSvm)
print('Akurasi Tree : ',akurasiTree)
print('Akurasi Random : ',akurasiRandom)

from sklearn.metrics import confusion_matrix
y_true = yTest
y_pred = gb.predict(xTest)
print(confusion_matrix(y_true, y_pred))

print(yTest.value_counts())

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Learning rate:  0.05
Akurasi Gradient Boosting :  0.9032258064516129
Learning rate:  0.1
Akurasi Gradient Boosting :  0.9032258064516129
Learning rate:  0.25
Akurasi Gradient Boosting :  0.9032258064516129
Learning rate:  0.5
Akurasi Gradient Boosting :  0.9032258064516129
Learning rate:  0.75
Akurasi Gradient Boosting :  0.9032258064516129
Learning rate:  1
Akurasi Gradient Boosting :  0.9032258064516129
Akurasi SVM :  (0.7717391304347826, 0.3870967741935484)
Akurasi Tree :  (1.0, 0.1935483870967742)
Akurasi Random :  0.3548387096774194
[[ 8  0  0  0]
 [ 0 10  2  0]
 [ 1  0  8  0]
 [ 0  0  0  2]]
2    12
3     9
1     8
4     2
Name: diagnosis, dtype: int64


In [42]:
df.head()

,umur,jenis_kelamin,tekanan_darah,gula_darah,thalach,exang,thal,diagnosis
0,32,1,95,0,127,0,0,1
1,34,1,115,0,154,0,0,1
2,35,1,0,0,130,1,7,3
3,36,1,110,0,125,1,6,1
4,38,0,105,0,166,0,0,2


In [14]:
#from werkzeug.wrappers import Request, Response
from flask import Flask
from flask import request
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import GridSearchCV
app = Flask(__name__)

@app.route("/")
def hello():
    return 'a'


if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Oct/2019 05:07:41] "GET / HTTP/1.1" 200 -
